<a href="https://colab.research.google.com/github/JayeshHarjule/LetsUpgradeAI-ML/blob/master/Project2_Health%20Anomaly%20Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import Python Packages**

In [3]:
import os
import sys
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from scipy.io import loadmat
!pip install pyod


     |████████████████████████████████| 102kB 4.1MB/s 
     |████████████████████████████████| 2.1MB 7.1MB/s 
  Created wheel for pyod: filename=pyod-0.8.6-cp36-none-any.whl size=112146 sha256=fbfc85817f8ec9590ae6c849aab57d4035ca189756705ebf474bbb8bc73923f9
  Stored in directory: /root/.cache/pip/wheels/48/95/81/2c6f52c3d366e7b5ef4770550f8bf0764257c66fbe64deecdb
  Created wheel for combo: filename=combo-0.1.2-cp36-none-any.whl size=42028 sha256=dc008cd94bfcf1063ce8980307611ab3d8c8152752225acf92bad2f512a3d0af
  Stored in directory: /root/.cache/pip/wheels/01/d9/bf/d1a371a5f0844cd8a53c04c14daa89974c93f429dda9dceb86
  Created wheel for suod: filename=suod-0.0.6-cp36-none-any.whl size=2154759 sha256=c581464d942f4dfbf38e6d0fe2ff0dded528b7c8331bb271c407c6ea43918c87
  Stored in directory: /root/.cache/pip/wheels/c5/d7/c1/6c778aee7fccfe3c054ea9bab92c5994ae3a0f6bba7078541e
Successfully built pyod combo suod


# **Import Pyod Packages & the methods**

In [27]:
import pyod
#Linear Model Based
from pyod.models.pca import PCA
from pyod.models.mcd import MCD
from pyod.models.ocsvm import OCSVM
#Proximity Based
from pyod.models.lof import LOF
from pyod.models.cblof import CBLOF
from pyod.models.knn import KNN
from pyod.models.hbos import HBOS
#Probability Based
from pyod.models.abod import ABOD
#Ensemble Based OD
from pyod.models.iforest import IForest
from pyod.models.feature_bagging import FeatureBagging

# **Import Metrics Packages**

In [28]:
from pyod.utils.utility import standardizer
from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score

# **Define data file and read X and Y**

In [7]:
mat_file_list = ['arrhythmia.mat',
                 'cardio.mat',
                 'glass.mat',
                 'ionosphere.mat',
                 'letter.mat',
                 'lympho.mat',
                 'mnist.mat',
                 'musk.mat',
                 'optdigits.mat',
                 'pendigits.mat',
                 'pima.mat',
                 'satellite.mat',
                 'satimage-2.mat',
                 'shuttle.mat',
                 'vertebral.mat',
                 'vowels.mat',
                 'wbc.mat']

In [29]:
mat_file_list

['arrhythmia.mat',
 'cardio.mat',
 'glass.mat',
 'ionosphere.mat',
 'letter.mat',
 'lympho.mat',
 'mnist.mat',
 'musk.mat',
 'optdigits.mat',
 'pendigits.mat',
 'pima.mat',
 'satellite.mat',
 'satimage-2.mat',
 'shuttle.mat',
 'vertebral.mat',
 'vowels.mat',
 'wbc.mat']

# **How to load Mat File**

In [30]:
from scipy.io import loadmat
data=loadmat('/data/cardio.mat')
data

{'X': array([[ 0.00491231,  0.69319077, -0.20364049, ...,  0.23149795,
         -0.28978574, -0.49329397],
        [ 0.11072935, -0.07990259, -0.20364049, ...,  0.09356344,
         -0.25638541, -0.49329397],
        [ 0.21654639, -0.27244466, -0.20364049, ...,  0.02459619,
         -0.25638541,  1.14001753],
        ...,
        [-0.41835583, -0.91998844, -0.16463485, ..., -1.49268341,
          0.24461959, -0.49329397],
        [-0.41835583, -0.91998844, -0.15093411, ..., -1.42371616,
          0.14441859, -0.49329397],
        [-0.41835583, -0.91998844, -0.20364049, ..., -1.28578165,
          3.58465295, -0.49329397]]),
 '__globals__': [],
 '__header__': b'MATLAB 5.0 MAT-file, written by Octave 3.8.0, 2014-12-18 10:48:09 UTC',
 '__version__': '1.0',
 'y': array([[0.],
        [0.],
        [0.],
        ...,
        [1.],
        [1.],
        [1.]])}

In [31]:
len(data)

5

In [32]:
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'X', 'y'])

In [33]:
data.values()

dict_values([b'MATLAB 5.0 MAT-file, written by Octave 3.8.0, 2014-12-18 10:48:09 UTC', '1.0', [], array([[ 0.00491231,  0.69319077, -0.20364049, ...,  0.23149795,
        -0.28978574, -0.49329397],
       [ 0.11072935, -0.07990259, -0.20364049, ...,  0.09356344,
        -0.25638541, -0.49329397],
       [ 0.21654639, -0.27244466, -0.20364049, ...,  0.02459619,
        -0.25638541,  1.14001753],
       ...,
       [-0.41835583, -0.91998844, -0.16463485, ..., -1.49268341,
         0.24461959, -0.49329397],
       [-0.41835583, -0.91998844, -0.15093411, ..., -1.42371616,
         0.14441859, -0.49329397],
       [-0.41835583, -0.91998844, -0.20364049, ..., -1.28578165,
         3.58465295, -0.49329397]]), array([[0.],
       [0.],
       [0.],
       ...,
       [1.],
       [1.],
       [1.]])])

# **Input(Independent) Feature Shape in Mat file format**

In [34]:
type(data['X']),data['X'].shape

(numpy.ndarray, (1831, 21))

# **Dependent/ Target /Output Feature shape**

In [35]:
type(data['y']),data['y'].shape

(numpy.ndarray, (1831, 1))

In [17]:
df_columns = ['Data', '#Samples', '# Dimensions', 'Outlier Perc',
              'ABOD', 'CBLOF', 'FB', 'HBOS', 'IForest', 'KNN', 'LOF', 'MCD',
              'OCSVM', 'PCA']

# **ROC Performance evulotion table**

In [36]:
roc_df = pd.DataFrame(columns=df_columns)

In [37]:
roc_df

,Data,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA


# **precision_n_scores - Performance evulotion table**

In [38]:
prn_df = pd.DataFrame(columns=df_columns)
prn_df

,Data,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA


# **Time dataframe**

In [39]:
time_df = pd.DataFrame(columns=df_columns)
time_df

,Data,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA


# **Exploring All Mat files**

In [41]:
from time import time
random_state = np.random.RandomState(42)

for mat_file in mat_file_list:
  print('\n... Proceesing the data', mat_file, '...')
  mat = loadmat(os.path.join('/data', mat_file))

  X = mat['X']
  y = mat['y'].ravel()
  outliers_fraction = np.count_nonzero(y) / len(y)
  outliers_percentage = round(outliers_fraction * 100, ndigits=4)

  # construct containers for saving results
  roc_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
  prn_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
  time_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]

  # 60% data for training and 40% for testing
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4,random_state=random_state)

  # standardizing data for processing
  X_train_norm, X_test_norm = standardizer(X_train, X_test)

  classifiers = {
      'Angle-based Outlier Detector (ABOD)': ABOD(contamination=outliers_fraction),
      'Cluster-based Local Outlier Factor': CBLOF(contamination=outliers_fraction, check_estimator=False,random_state=random_state),
      'Feature Bagging': FeatureBagging(contamination=outliers_fraction, random_state=random_state),
      'Histogram-base Outlier Detection (HBOS)': HBOS(contamination=outliers_fraction),
      'Isolation Forest': IForest(contamination=outliers_fraction, random_state=random_state),
      'K Nearest Neighbors (KNN)': KNN(contamination=outliers_fraction),
      'Local Outlier Factor (LOF)': LOF(contamination=outliers_fraction),
      'Minimum Covariance Determinant (MCD)': MCD(contamination=outliers_fraction, random_state=random_state),
      'One-class SVM (OCSVM)': OCSVM(contamination=outliers_fraction),
      'Principal Component Analysis (PCA)': PCA(contamination=outliers_fraction, random_state=random_state), 
  }

  for clf_name, clf in classifiers.items():
    t0=time()
    clf.fit(X_train_norm)
    test_scores = clf.decision_function(X_test_norm)
    t1 = time()
    duration = round(t1 - t0, ndigits=4)
    time_list.append(duration)

    roc = round(roc_auc_score(y_test, test_scores), ndigits=4)
    prn = round(precision_n_scores(y_test, test_scores), ndigits=4)

    print('{clf_name} ROC:{roc}, precision @ rank n:{prn},''execution time: {duration}s'.format(clf_name=clf_name, roc=roc, prn=prn, duration=duration))

    roc_list.append(roc)
    prn_list.append(prn)
  
  temp_df = pd.DataFrame(time_list).transpose()
  temp_df.columns = df_columns
  time_df = pd.concat([time_df, temp_df], axis=0)

  temp_df = pd.DataFrame(roc_list).transpose()
  temp_df.columns = df_columns
  roc_df = pd.concat([roc_df, temp_df], axis=0)

  temp_df = pd.DataFrame(prn_list).transpose()
  temp_df.columns = df_columns
  prn_df = pd.concat([prn_df, temp_df], axis=0)



... Proceesing the data arrhythmia.mat ...
Angle-based Outlier Detector (ABOD) ROC:0.7687, precision @ rank n:0.3571,execution time: 1.4218s
Cluster-based Local Outlier Factor ROC:0.7684, precision @ rank n:0.4643,execution time: 1.518s
Feature Bagging ROC:0.7799, precision @ rank n:0.5,execution time: 0.6749s
Histogram-base Outlier Detection (HBOS) ROC:0.8511, precision @ rank n:0.5714,execution time: 1.8561s
Isolation Forest ROC:0.8527, precision @ rank n:0.5714,execution time: 0.4166s
K Nearest Neighbors (KNN) ROC:0.782, precision @ rank n:0.5,execution time: 0.0991s
Local Outlier Factor (LOF) ROC:0.7787, precision @ rank n:0.4643,execution time: 0.0803s


/usr/local/lib/python3.6/dist-packages/sklearn/covariance/_robust_covariance.py:644: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "


Minimum Covariance Determinant (MCD) ROC:0.8228, precision @ rank n:0.4286,execution time: 2.0306s
One-class SVM (OCSVM) ROC:0.7986, precision @ rank n:0.5,execution time: 0.0599s
Principal Component Analysis (PCA) ROC:0.8, precision @ rank n:0.5,execution time: 0.0698s

... Proceesing the data cardio.mat ...
Angle-based Outlier Detector (ABOD) ROC:0.5763, precision @ rank n:0.1875,execution time: 0.3733s
Cluster-based Local Outlier Factor ROC:0.8221, precision @ rank n:0.4844,execution time: 0.2429s
Feature Bagging ROC:0.4879, precision @ rank n:0.1406,execution time: 0.8111s
Histogram-base Outlier Detection (HBOS) ROC:0.8453, precision @ rank n:0.4688,execution time: 0.0082s
Isolation Forest ROC:0.9414, precision @ rank n:0.5,execution time: 0.4043s
K Nearest Neighbors (KNN) ROC:0.6959, precision @ rank n:0.2812,execution time: 0.1374s
Local Outlier Factor (LOF) ROC:0.4715, precision @ rank n:0.125,execution time: 0.1014s


/usr/local/lib/python3.6/dist-packages/sklearn/covariance/_robust_covariance.py:644: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "


Minimum Covariance Determinant (MCD) ROC:0.8778, precision @ rank n:0.3906,execution time: 1.4049s
One-class SVM (OCSVM) ROC:0.9507, precision @ rank n:0.5938,execution time: 0.1524s
Principal Component Analysis (PCA) ROC:0.9638, precision @ rank n:0.6875,execution time: 0.0048s

... Proceesing the data glass.mat ...
Angle-based Outlier Detector (ABOD) ROC:0.7104, precision @ rank n:0.25,execution time: 0.0627s
Cluster-based Local Outlier Factor ROC:0.8506, precision @ rank n:0.25,execution time: 0.0575s
Feature Bagging ROC:0.7043, precision @ rank n:0.25,execution time: 0.0395s
Histogram-base Outlier Detection (HBOS) ROC:0.6524, precision @ rank n:0.0,execution time: 0.0047s
Isolation Forest ROC:0.7195, precision @ rank n:0.25,execution time: 0.2771s
K Nearest Neighbors (KNN) ROC:0.7805, precision @ rank n:0.25,execution time: 0.0079s
Local Outlier Factor (LOF) ROC:0.7774, precision @ rank n:0.25,execution time: 0.0042s
Minimum Covariance Determinant (MCD) ROC:0.7165, precision @ rank

...

Minimum Covariance Determinant (MCD) ROC:0.863, precision @ rank n:0.3973,execution time: 5.5319s
One-class SVM (OCSVM) ROC:0.8417, precision @ rank n:0.3801,execution time: 5.852s
Principal Component Analysis (PCA) ROC:0.8396, precision @ rank n:0.3767,execution time: 0.1273s

... Proceesing the data musk.mat ...
Angle-based Outlier Detector (ABOD) ROC:0.0809, precision @ rank n:0.0333,execution time: 2.3381s
Cluster-based Local Outlier Factor ROC:1.0, precision @ rank n:1.0,execution time: 0.4152s
Feature Bagging ROC:0.5228, precision @ rank n:0.1667,execution time: 14.6824s
Histogram-base Outlier Detection (HBOS) ROC:0.9999, precision @ rank n:0.9667,execution time: 0.063s
Isolation Forest ROC:0.9996, precision @ rank n:0.9333,execution time: 0.8859s
K Nearest Neighbors (KNN) ROC:0.7348, precision @ rank n:0.2333,execution time: 1.9671s
Local Outlier Factor (LOF) ROC:0.5323, precision @ rank n:0.1333,execution time: 1.8512s
Minimum Covariance Determinant (MCD) ROC:1.0, precision @ r

/usr/local/lib/python3.6/dist-packages/sklearn/covariance/_robust_covariance.py:644: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "


Minimum Covariance Determinant (MCD) ROC:0.3486, precision @ rank n:0.0,execution time: 2.6778s
One-class SVM (OCSVM) ROC:0.4972, precision @ rank n:0.0,execution time: 1.9214s
Principal Component Analysis (PCA) ROC:0.504, precision @ rank n:0.0,execution time: 0.0441s

... Proceesing the data pendigits.mat ...
Angle-based Outlier Detector (ABOD) ROC:0.7008, precision @ rank n:0.0308,execution time: 1.3982s
Cluster-based Local Outlier Factor ROC:0.9609, precision @ rank n:0.3077,execution time: 0.4131s
Feature Bagging ROC:0.4687, precision @ rank n:0.0462,execution time: 3.941s
Histogram-base Outlier Detection (HBOS) ROC:0.9294, precision @ rank n:0.2615,execution time: 0.0104s
Isolation Forest ROC:0.9422, precision @ rank n:0.2769,execution time: 0.7094s
K Nearest Neighbors (KNN) ROC:0.7602, precision @ rank n:0.0462,execution time: 0.5329s
Local Outlier Factor (LOF) ROC:0.481, precision @ rank n:0.0462,execution time: 0.4772s
Minimum Covariance Determinant (MCD) ROC:0.8271, precision

/usr/local/lib/python3.6/dist-packages/sklearn/covariance/_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-49.541490230701072 > -75.594915465958152). You may want to try with a higher value of support_fraction (current value: 0.503).
  RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/covariance/_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-50.022992269210988 > -76.128980282328513). You may want to try with a higher value of support_fraction (current value: 0.503).
  RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/covariance/_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-48.588090860494674 > -73.972064354883088). You may want to try with a higher value of support_fraction (current value: 0.503).
  RuntimeWarning)
/usr/local/lib/p

Minimum Covariance Determinant (MCD) ROC:0.9903, precision @ rank n:0.7534,execution time: 26.4235s
One-class SVM (OCSVM) ROC:0.9922, precision @ rank n:0.9553,execution time: 66.5636s
Principal Component Analysis (PCA) ROC:0.9902, precision @ rank n:0.9503,execution time: 0.0332s

... Proceesing the data vertebral.mat ...
Angle-based Outlier Detector (ABOD) ROC:0.2797, precision @ rank n:0.0,execution time: 0.0581s
Cluster-based Local Outlier Factor ROC:0.3908, precision @ rank n:0.0,execution time: 0.0514s
Feature Bagging ROC:0.3027, precision @ rank n:0.0,execution time: 0.0398s
Histogram-base Outlier Detection (HBOS) ROC:0.2695, precision @ rank n:0.0,execution time: 0.002s
Isolation Forest ROC:0.3576, precision @ rank n:0.0,execution time: 0.2884s
K Nearest Neighbors (KNN) ROC:0.318, precision @ rank n:0.0,execution time: 0.0087s
Local Outlier Factor (LOF) ROC:0.318, precision @ rank n:0.0,execution time: 0.0032s
Minimum Covariance Determinant (MCD) ROC:0.3308, precision @ rank n:

In [42]:
roc_df

,Data,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA
0,arrhythmia,452,274,14.6018,0.7687,0.7684,0.7799,0.8511,0.8527,0.782,0.7787,0.8228,0.7986,0.8
0,cardio,1831,21,9.6122,0.5763,0.8221,0.4879,0.8453,0.9414,0.6959,0.4715,0.8778,0.9507,0.9638
0,glass,214,9,4.2056,0.7104,0.8506,0.7043,0.6524,0.7195,0.7805,0.7774,0.7165,0.6189,0.622
0,ionosphere,351,33,35.8974,0.9004,0.8952,0.8933,0.5195,0.8309,0.9134,0.8989,0.9399,0.8372,0.7971
0,letter,1600,32,6.25,0.8465,0.7423,0.866,0.5728,0.5778,0.845,0.8409,0.7499,0.5744,0.48
0,lympho,148,18,4.0541,0.9382,0.9709,0.9673,0.9964,0.9855,0.9636,0.9636,0.9164,0.9636,0.9818
0,mnist,7603,100,9.2069,0.7813,0.8447,0.7259,0.5675,0.7801,0.8409,0.7085,0.863,0.8417,0.8396
0,musk,3062,166,3.1679,0.0809,1,0.5228,0.9999,0.9996,0.7348,0.5323,1,1,1
0,optdigits,5216,64,2.8758,0.4428,0.7852,0.4641,0.8822,0.5764,0.3824,0.4584,0.3486,0.4972,0.504
0,pendigits,6870,16,2.2707,0.7008,0.9609,0.4687,0.9294,0.9422,0.7602,0.481,0.8271,0.93,0.9332


In [43]:
prn_df

,Data,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA
0,arrhythmia,452,274,14.6018,0.3571,0.4643,0.5,0.5714,0.5714,0.5,0.4643,0.4286,0.5,0.5
0,cardio,1831,21,9.6122,0.1875,0.4844,0.1406,0.4688,0.5,0.2812,0.125,0.3906,0.5938,0.6875
0,glass,214,9,4.2056,0.25,0.25,0.25,0,0.25,0.25,0.25,0,0.25,0.25
0,ionosphere,351,33,35.8974,0.8214,0.8036,0.75,0.3393,0.6607,0.8393,0.75,0.8571,0.7143,0.5893
0,letter,1600,32,6.25,0.275,0.175,0.4,0.125,0.05,0.3,0.325,0.075,0.1,0.05
0,lympho,148,18,4.0541,0.4,0.6,0.6,0.8,0.6,0.6,0.6,0.6,0.6,0.8
0,mnist,7603,100,9.2069,0.3562,0.4007,0.3664,0.1199,0.2979,0.4144,0.339,0.3973,0.3801,0.3767
0,musk,3062,166,3.1679,0.0333,1,0.1667,0.9667,0.9333,0.2333,0.1333,0.9667,1,1
0,optdigits,5216,64,2.8758,0.0161,0,0.0484,0.2581,0.0161,0,0.0484,0,0,0
0,pendigits,6870,16,2.2707,0.0308,0.3077,0.0462,0.2615,0.2769,0.0462,0.0462,0.0615,0.2923,0.3385


In [44]:
time_df

,Data,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA
0,arrhythmia,452,274,14.6018,1.4218,1.518,0.6749,1.8561,0.4166,0.0991,0.0803,2.0306,0.0599,0.0698
0,cardio,1831,21,9.6122,0.3733,0.2429,0.8111,0.0082,0.4043,0.1374,0.1014,1.4049,0.1524,0.0048
0,glass,214,9,4.2056,0.0627,0.0575,0.0395,0.0047,0.2771,0.0079,0.0042,0.0675,0.0032,0.0021
0,ionosphere,351,33,35.8974,0.0911,0.0536,0.0743,0.0093,0.3137,0.0179,0.0084,0.1323,0.0084,0.0039
0,letter,1600,32,6.25,0.3627,0.1783,0.8193,0.0102,0.4049,0.13,0.1009,3.4335,0.1404,0.0102
0,lympho,148,18,4.0541,0.0448,0.0607,0.032,0.0046,0.3141,0.0062,0.003,0.112,0.0024,0.0024
0,mnist,7603,100,9.2069,7.9481,1.1318,56.8401,0.0541,1.2634,7.4309,7.1808,5.5319,5.852,0.1273
0,musk,3062,166,3.1679,2.3381,0.4152,14.6824,0.063,0.8859,1.9671,1.8512,23.9882,1.5222,0.1213
0,optdigits,5216,64,2.8758,2.6479,0.558,16.0543,0.0323,0.8756,2.0652,1.9029,2.6778,1.9214,0.0441
0,pendigits,6870,16,2.2707,1.3982,0.4131,3.941,0.0104,0.7094,0.5329,0.4772,4.2557,1.3634,0.0104
